ParamNB is a small library that represents Parameters (defined using the [Param library](https://github.com/ioam/param)) graphically in a Jupyter notebook using [ipywidgets](https://ipywidgets.readthedocs.io).

To use ParamNB, first declare a Parameterized class with some Parameters:

In [ ]:
import param

def hello(x):
    print("Hello %s" % x)
    
class Example(param.Parameterized):
    """An example Parameterized class"""
    
    x                       = param.Parameter(default=3.14,doc="X position")
    y                       = param.Parameter(default="Not editable",constant=True)
    string_value            = param.String(default="str",doc="A string")
    num_int                 = param.Integer(50000,bounds=(-200,100000))
    unbounded_int           = param.Integer(23)
    float_with_hard_bounds  = param.Number(8.2,bounds=(7.5,10))
    float_with_soft_bounds  = param.Number(0.5,bounds=(0,None),softbounds=(0,2))
    unbounded_float         = param.Number(30.01)
    hidden_parameter        = param.Number(2.718,precedence=-1)
    boolean                 = param.Boolean(True, doc="A sample Boolean parameter")
    select_string           = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
    select_fn               = param.ObjectSelector(default=list,objects=[list,set,dict])
    int_list                = param.ListSelector(default=[3,5], objects=[1,3,5,7,9],precedence=0.5)
    single_file             = param.FileSelector(path='../*/*.py*',precedence=0.5)
    multiple_files          = param.MultiFileSelector(path='../*/*.py?',precedence=0.5)
    msg                     = param.Action(hello, doc="""Print a message.""",precedence=0.7)
    
Example.num_int

Declaring Parameters depends only on the separate Param library.  Param is a pure Python library with no dependencies, which makes it easy to include in any code without tying it to a particular GUI toolkit or to the Jupyter notebook.  And as you can see, the declarations above focus on the semantic content of your parameters, without worrying about how they might appear later in a GUI or notebook.

Then once you are in a notebook environment, you can import ParamNB so that you can display and edit these parameter values at the class level by wrapping the Parameterized class in a ``paramnb.Widgets`` object (and similarly for objects).  

By default, the widgets object simply displays the values and allows them to be edited, which will work well if running the notebook cell by cell -- when you get to this cell, edit the values as you like, then move on.  If you do `Run All` in the notebook instead, there will be no opportunity to edit the values before they are used elsewhere in the notebook, but you can edit them and then do `Run All Below` in the notebook to see the results.  Some examples:

In [ ]:
import paramnb

paramnb.Widgets(Example)

If you now look at the values of the Parameters at the class level (re-executing this cell if needed), you can see that they have changed to whatever was selected interactively above:

In [ ]:
Example.unbounded_int

In [ ]:
Example.num_int

In [ ]:
#Example.print_param_defaults() # see all parameter values

Subsequent cells can now refer to these values, e.g. as `Example.num_int`.  

Having to work cell by cell or re-run the notebook manually can be awkward, e.g. when building dashboards that hide the notebook user interface.  In order to provide "live" or dynamic updating, ParamNB also allows you to control code execution in various ways.  First, you can define *what* code will be executed:

* `callback`=*callable*: User-defined function to call
* `next_n`=*n*: zero by default, but if set to e.g. 2, will execute the subsequent 2 cells of the notebook

You can also define *when* the code will be executed:

* `button=False`: the default; the specified code will be executed whenever a widget value is changed
* `button=True`: Provide a button to control code execution, so that multiple widgets can be adjusted and code is updated only when the button is pushed.

`button=False` can provide a smooth, dynamic user experience, with text or plots updating as a slider is dragged.  However, this approach will work well only for relatively inexpensive operations, because the code may get executed multiple times for a single interaction with a widget, and there may be multiple widgets that need adjusting.  Thus for computationally intensive code, you may prefer to set `button=True`.  

Example of dynamic updating:

In [ ]:
class Example2(param.Parameterized):
    num1 = param.Number(3.14,bounds=(0.0,10.0))
    number2 = param.Integer(2,bounds=(0,5))

paramnb.Widgets(Example2,next_n=1)

In [ ]:
Example2.num1, Example2.number2

Notice that the number of the above cell increases every time you drag a slider, because that cell is being re-executed.  

Example of updating on the "Run" button press:

In [ ]:
paramnb.Widgets(Example2,button=True,callback=hello,next_n=1)

In [ ]:
Example2.num1, Example2.number2

Here, the cell above changes its number (and output value) only when the "Run 1" button is pressed in the previous cell.  The supplied callback is also executed.

Together, these features make it simple to add interactive controls in Jupyter notebooks: just declare your parameters wherever they are going to be used, using the Param library (pure Python, zero dependencies), then add a `Widgets()` declaration in your notebook wherever you want to be able to modify those values interactively.  That way your main code can be fully independent of any GUI or notebook display, while your notebooks can simply expose the parameters declared in your main code, without duplicating them or relying on specific details of that code.  So you can now have full interactivity without tying yourself to any particular user interface or GUI library.

Have fun!